# Normalization and Tuning Neural Networks - Lab

## Introduction

For this lab on initialization and optimization, let's look at a slightly different type of neural network. This time, we will not perform a classification task as we've done before (Santa vs not santa, bank complaint types), but we'll look at a linear regression problem.

We can just as well use deep learning networks for linear regression as for a classification problem. Do note that getting regression to work with neural networks is a hard problem because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$, and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case!

## Objectives
You will be able to:
* Build a nueral network using keras
* Normalize your data to assist algorithm convergence
* Implement and observe the impact of various initialization techniques

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from keras import optimizers
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## Loading the data

The data we'll be working with is data related to facebook posts published during the year of 2014 on the Facebook's page of a renowned cosmetics brand.  It includes 7 features known prior to post publication, and 12 features for evaluating the post impact. What we want to do is make a predictor for the number of "likes" for a post, taking into account the 7 features prior to posting.

First, let's import the data set and delete any rows with missing data. Afterwards, briefly preview the data.

In [34]:
#Your code here; load the dataset and drop rows with missing values. Then preview the data.
df = pd.read_csv('dataset_Facebook.csv', sep=';', header=0)
df = df.dropna()
df.head()


,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


## Initialization

## Normalize the Input Data

Let's look at our input data. We'll use the 7 first columns as our predictors. We'll do the following two things:
- Normalize the continuous variables --> you can do this using `np.mean()` and `np.std()`
- Make dummy variables of the categorical variables (you can do this by using `pd.get_dummies`)

We only count "Category" and "Type" as categorical variables. Note that you can argue that "Post month", "Post Weekday" and "Post Hour" can also be considered categories, but we'll just treat them as being continuous for now.

You'll then use these to define X and Y. 

To summarize, X will be:
* Page total likes
* Post Month
* Post Weekday
* Post Hour
* Paid
along with dummy variables for:
* Type
* Category


Be sure to normalize your features by subtracting the mean and dividing by the standard deviation.  

Finally, y will simply be the "like" column.

In [35]:
data = df
data.head()

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


In [69]:
data["Post Hour"].unique()

array([ 3, 10,  9,  5, 12, 11,  2,  4, 13,  8,  7,  6,  1, 14, 23, 22, 15,
       20, 19, 18, 17, 16], dtype=int64)

In [36]:
df.columns

Index(['Page total likes', 'Type', 'Category', 'Post Month', 'Post Weekday',
       'Post Hour', 'Paid', 'Lifetime Post Total Reach',
       'Lifetime Post Total Impressions', 'Lifetime Engaged Users',
       'Lifetime Post Consumers', 'Lifetime Post Consumptions',
       'Lifetime Post Impressions by people who have liked your Page',
       'Lifetime Post reach by people who like your Page',
       'Lifetime People who have liked your Page and engaged with your post',
       'comment', 'like', 'share', 'Total Interactions'],
      dtype='object')

In [37]:
X0 = data["Page total likes"]
X1 = data["Type"]
X2 = data["Category"]
X3 = data["Post Month"]
X4 = data["Post Weekday"]
X5 = data["Post Hour"]
X6 = data["Paid"]

## standardize/categorize
X0= (X0-np.mean(X0))/(np.std(X0))
dummy_X1= pd.get_dummies(X1)
dummy_X2= pd.get_dummies(X2)
X3= (X3-np.mean(X3))/(np.std(X3))
X4= (X4-np.mean(X4))/(np.std(X4))
X5= (X5-np.mean(X5))/(np.std(X5))

X = pd.concat([X0, dummy_X1, dummy_X2, X3, X4, X5, X6], axis=1)

In [38]:
X.head()

,Page total likes,Link,Photo,Status,Video,1,2,3,Post Month,Post Weekday,Post Hour,Paid
0,1.00496,0,1,0,0,0,1,0,1.506154,-0.065724,-1.105878,0.0
1,1.00496,0,0,1,0,0,1,0,1.506154,-0.558655,0.492065,0.0
2,1.00496,0,1,0,0,0,0,1,1.506154,-0.558655,-1.105878,0.0
3,1.00496,0,1,0,0,0,1,0,1.506154,-1.051585,0.492065,1.0
4,1.00496,0,1,0,0,0,1,0,1.506154,-1.051585,-1.105878,0.0


In [39]:
Y = data["like"]
Y.head()

0      79.0
1     130.0
2      66.0
3    1572.0
4     325.0
Name: like, dtype: float64

Our data is fairly small. Let's just split the data up in a training set and a validation set!  The next three code blocks are all provided for you; have a quick review but not need to make edits!

In [40]:
#Code provided; defining training and validation sets
data_clean = pd.concat([X, Y], axis=1)
np.random.seed(123)
train, validation = train_test_split(data_clean, test_size=0.2)

X_val = validation.iloc[:,0:12]
Y_val = validation.iloc[:,12]
X_train = train.iloc[:,0:12]
Y_train = train.iloc[:,12]

In [45]:
X_train.head()

,Page total likes,Link,Photo,Status,Video,1,2,3,Post Month,Post Weekday,Post Hour,Paid
208,0.557700,0,1,0,0,0,0,1,0.294376,1.413068,-1.105878,0.0
290,0.150656,0,1,0,0,1,0,0,-0.311513,-0.065724,-0.192768,0.0
286,0.183335,0,1,0,0,1,0,0,-0.311513,0.427207,0.720342,1.0
0,1.004960,0,1,0,0,0,1,0,1.506154,-0.065724,-1.105878,0.0
401,-0.943093,0,1,0,0,1,0,0,-0.917402,-0.558655,1.176897,1.0


In [47]:
#Code provided; building an initial model
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=1)

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 463us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/100
396/396 [==============================] - 0s 48us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/100
396/396 [==============================] - 0s 65us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/100
396/396 [==============================] - 0s 58us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/100
396/396 [==============================] - 0s 63us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/100
396/396 [==============================] - 0s 48us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/100
396/396 [=====================

396/396 [==============================] - 0s 83us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/100
396/396 [==============================] - 0s 78us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/100
396/396 [==============================] - 0s 83us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/100
396/396 [==============================] - 0s 81us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/100
396/396 [==============================] - 0s 86us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/100
396/396 [==============================] - 0s 83us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 60/100
396/396 [==============================] - 0s 91us/step - loss: nan - mean_squared

In [6]:
#Code provided; previewing the loss through successive epochs
hist.history['loss'][:10]

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

Did you see what happend? all the values for training and validation loss are "nan". There could be several reasons for that, but as we already mentioned there is likely a vanishing or exploding gradient problem. recall that we normalized out inputs. But how about the outputs? Let's have a look.

In [7]:
Y_train.head()

208     54.0
290     23.0
286     15.0
0       79.0
401    329.0
Name: like, dtype: float64

Yes, indeed. We didn't normalize them and we should, as they take pretty high values. Let
s rerun the model but make sure that the output is normalized as well!

## Normalizing the output

Normalize Y as you did X by subtracting the mean and dividing by the standard deviation. Then, resplit the data into training and validation sets as we demonstrated above, and retrain a new model using your normalized X and Y data.

In [49]:
#Your code here: redefine Y after normalizing the data.
Y = (data['like'] - np.mean(data['like'])) / np.std(data['like'])

In [59]:
Y.head()

0   -0.309011
1   -0.151644
2   -0.349124
3    4.297815
4    0.450051
Name: like, dtype: float64

In [70]:
Y.describe()

count    4.950000e+02
mean    -1.794300e-17
std      1.001012e+00
min     -5.527742e-01
25%     -3.768941e-01
50%     -2.411269e-01
75%      2.732173e-02
max      1.540604e+01
Name: like, dtype: float64

In [50]:
#Your code here; create training and validation sets as before. Use random seed 123.
data_clean = pd.concat([X, Y], axis=1)
np.random.seed(123)
train, validation = train_test_split(data_clean, test_size=0.2)

X_val = validation.iloc[:,0:12]
Y_val = validation.iloc[:,12]
X_train = train.iloc[:,0:12]
Y_train = train.iloc[:,12]

In [60]:
data_clean.head()

,Page total likes,Link,Photo,Status,Video,1,2,3,Post Month,Post Weekday,Post Hour,Paid,like
0,1.00496,0,1,0,0,0,1,0,1.506154,-0.065724,-1.105878,0.0,-0.309011
1,1.00496,0,0,1,0,0,1,0,1.506154,-0.558655,0.492065,0.0,-0.151644
2,1.00496,0,1,0,0,0,0,1,1.506154,-0.558655,-1.105878,0.0,-0.349124
3,1.00496,0,1,0,0,0,1,0,1.506154,-1.051585,0.492065,1.0,4.297815
4,1.00496,0,1,0,0,0,1,0,1.506154,-1.051585,-1.105878,0.0,0.450051


In [53]:
#Your code here; rebuild a simple model using a relu layer followed by a linear layer. (See our code snippet above!)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=1)

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 690us/step - loss: 1.3721 - mean_squared_error: 1.3721 - val_loss: 1.1379 - val_mean_squared_error: 1.1379
Epoch 2/100
396/396 [==============================] - 0s 60us/step - loss: 1.2270 - mean_squared_error: 1.2270 - val_loss: 1.0890 - val_mean_squared_error: 1.0890
Epoch 3/100
396/396 [==============================] - 0s 63us/step - loss: 1.1753 - mean_squared_error: 1.1753 - val_loss: 1.0659 - val_mean_squared_error: 1.0659
Epoch 4/100
396/396 [==============================] - 0s 55us/step - loss: 1.1450 - mean_squared_error: 1.1450 - val_loss: 1.0485 - val_mean_squared_error: 1.0485
Epoch 5/100
396/396 [==============================] - 0s 60us/step - loss: 1.1234 - mean_squared_error: 1.1234 - val_loss: 1.0366 - val_mean_squared_error: 1.0366
Epoch 6/100
396/396 [==============================] - 0s 55us/step - loss: 1.1054 - mean_squared_error: 1.1054 - val_loss: 1.0287 - v

396/396 [==============================] - 0s 78us/step - loss: 0.9844 - mean_squared_error: 0.9844 - val_loss: 0.9640 - val_mean_squared_error: 0.9640
Epoch 51/100
396/396 [==============================] - 0s 76us/step - loss: 0.9844 - mean_squared_error: 0.9844 - val_loss: 0.9637 - val_mean_squared_error: 0.9637
Epoch 52/100
396/396 [==============================] - 0s 76us/step - loss: 0.9840 - mean_squared_error: 0.9840 - val_loss: 0.9622 - val_mean_squared_error: 0.9622
Epoch 53/100
396/396 [==============================] - 0s 88us/step - loss: 0.9834 - mean_squared_error: 0.9834 - val_loss: 0.9653 - val_mean_squared_error: 0.9653
Epoch 54/100
396/396 [==============================] - 0s 78us/step - loss: 0.9830 - mean_squared_error: 0.9830 - val_loss: 0.9631 - val_mean_squared_error: 0.9631
Epoch 55/100
396/396 [==============================] - 0s 73us/step - loss: 0.9831 - mean_squared_error: 0.9831 - val_loss: 0.9527 - val_mean_squared_error: 0.9527
Epoch 56/100
396/396 [=

Epoch 100/100
396/396 [==============================] - 0s 73us/step - loss: 0.9727 - mean_squared_error: 0.9727 - val_loss: 0.9529 - val_mean_squared_error: 0.9529


Finally, let's recheck our loss function. Not only should it be populated with numerical data as opposed to null values, but we also should expect to see the loss function decreasing with successive epochs, demonstrating optimization!

In [54]:
hist.history['loss'][:10]

[1.3720854445539339,
 1.2270435284484515,
 1.1753324318413783,
 1.1450198280690895,
 1.123367585317053,
 1.1054006512417938,
 1.089552349815465,
 1.0777066149043315,
 1.0678775272887162,
 1.058149522753677]

Great! We have a converged model. With that, let's investigate how well the model performed with our good old friend, mean squarred error.

In [55]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)  

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

print("MSE_train:", MSE_train)
print("MSE_val:", MSE_val)

MSE_train: 0.9707142903228307
MSE_val: 0.952940722303891


## Using Weight Initializers

##  He Initialization

Let's try and use a weight initializer. In the lecture, we've seen the He normalizer, which initializes the weight vector to have an average 0 and a variance of 2/n, with $n$ the number of features feeding into a layer.

In [58]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, kernel_initializer= "he_normal",
                activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val),verbose=1)

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 907us/step - loss: 1.5102 - mean_squared_error: 1.5102 - val_loss: 1.2863 - val_mean_squared_error: 1.2863
Epoch 2/100
396/396 [==============================] - 0s 50us/step - loss: 1.2107 - mean_squared_error: 1.2107 - val_loss: 1.1489 - val_mean_squared_error: 1.1489
Epoch 3/100
396/396 [==============================] - 0s 48us/step - loss: 1.1401 - mean_squared_error: 1.1401 - val_loss: 1.0889 - val_mean_squared_error: 1.0889
Epoch 4/100
396/396 [==============================] - 0s 55us/step - loss: 1.1068 - mean_squared_error: 1.1068 - val_loss: 1.0574 - val_mean_squared_error: 1.0574
Epoch 5/100
396/396 [==============================] - 0s 53us/step - loss: 1.0850 - mean_squared_error: 1.0850 - val_loss: 1.0377 - val_mean_squared_error: 1.0377
Epoch 6/100
396/396 [==============================] - 0s 58us/step - loss: 1.0678 - mean_squared_error: 1.0678 - val_loss: 1.0239 - v

396/396 [==============================] - 0s 71us/step - loss: 0.9552 - mean_squared_error: 0.9552 - val_loss: 0.9396 - val_mean_squared_error: 0.9396
Epoch 51/100
396/396 [==============================] - 0s 73us/step - loss: 0.9541 - mean_squared_error: 0.9541 - val_loss: 0.9394 - val_mean_squared_error: 0.9394
Epoch 52/100
396/396 [==============================] - 0s 76us/step - loss: 0.9529 - mean_squared_error: 0.9529 - val_loss: 0.9366 - val_mean_squared_error: 0.9366
Epoch 53/100
396/396 [==============================] - 0s 78us/step - loss: 0.9530 - mean_squared_error: 0.9530 - val_loss: 0.9386 - val_mean_squared_error: 0.9386
Epoch 54/100
396/396 [==============================] - 0s 73us/step - loss: 0.9526 - mean_squared_error: 0.9526 - val_loss: 0.9418 - val_mean_squared_error: 0.9418
Epoch 55/100
396/396 [==============================] - 0s 71us/step - loss: 0.9512 - mean_squared_error: 0.9512 - val_loss: 0.9386 - val_mean_squared_error: 0.9386
Epoch 56/100
396/396 [=

396/396 [==============================] - 0s 73us/step - loss: 0.9297 - mean_squared_error: 0.9297 - val_loss: 0.9508 - val_mean_squared_error: 0.9508
Epoch 100/100
396/396 [==============================] - 0s 71us/step - loss: 0.9292 - mean_squared_error: 0.9292 - val_loss: 0.9477 - val_mean_squared_error: 0.9477


In [71]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [72]:
print(MSE_train)
print(MSE_val)

0.9266121377054887
0.947702462558325


The initializer does not really help us to decrease the MSE. We know that initializers can be particularly helpful in deeper networks, and our network isn't very deep. What if we use the `Lecun` initializer with a `tanh` activation?

## Lecun Initialization

In [73]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, 
                kernel_initializer= "lecun_normal", activation='tanh'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "sgd" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose=1)

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 1ms/step - loss: 1.2427 - mean_squared_error: 1.2427 - val_loss: 1.0645 - val_mean_squared_error: 1.0645
Epoch 2/100
396/396 [==============================] - 0s 53us/step - loss: 1.1424 - mean_squared_error: 1.1424 - val_loss: 1.0116 - val_mean_squared_error: 1.0116
Epoch 3/100
396/396 [==============================] - 0s 50us/step - loss: 1.0868 - mean_squared_error: 1.0868 - val_loss: 0.9843 - val_mean_squared_error: 0.9843
Epoch 4/100
396/396 [==============================] - 0s 60us/step - loss: 1.0533 - mean_squared_error: 1.0533 - val_loss: 0.9681 - val_mean_squared_error: 0.9681
Epoch 5/100
396/396 [==============================] - 0s 55us/step - loss: 1.0336 - mean_squared_error: 1.0336 - val_loss: 0.9608 - val_mean_squared_error: 0.9608
Epoch 6/100
396/396 [==============================] - 0s 53us/step - loss: 1.0158 - mean_squared_error: 1.0158 - val_loss: 0.9539 - val

396/396 [==============================] - 0s 65us/step - loss: 0.9455 - mean_squared_error: 0.9455 - val_loss: 0.9526 - val_mean_squared_error: 0.9526
Epoch 51/100
396/396 [==============================] - 0s 71us/step - loss: 0.9449 - mean_squared_error: 0.9449 - val_loss: 0.9523 - val_mean_squared_error: 0.9523
Epoch 52/100
396/396 [==============================] - 0s 78us/step - loss: 0.9428 - mean_squared_error: 0.9428 - val_loss: 0.9466 - val_mean_squared_error: 0.9466
Epoch 53/100
396/396 [==============================] - 0s 73us/step - loss: 0.9441 - mean_squared_error: 0.9441 - val_loss: 0.9496 - val_mean_squared_error: 0.9496
Epoch 54/100
396/396 [==============================] - 0s 73us/step - loss: 0.9445 - mean_squared_error: 0.9445 - val_loss: 0.9539 - val_mean_squared_error: 0.9539
Epoch 55/100
396/396 [==============================] - 0s 73us/step - loss: 0.9426 - mean_squared_error: 0.9426 - val_loss: 0.9465 - val_mean_squared_error: 0.9465
Epoch 56/100
396/396 [=

Epoch 100/100
396/396 [==============================] - 0s 65us/step - loss: 0.9321 - mean_squared_error: 0.9321 - val_loss: 0.9463 - val_mean_squared_error: 0.9463


In [74]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [75]:
print(MSE_train)
print(MSE_val)

0.9274741851539924
0.9463006320907383


Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impace of various optimization algorithms.

## RMSprop

In [76]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "rmsprop" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 1)

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 0s 1ms/step - loss: 1.2354 - mean_squared_error: 1.2354 - val_loss: 1.3119 - val_mean_squared_error: 1.3119
Epoch 2/100
396/396 [==============================] - 0s 50us/step - loss: 1.1875 - mean_squared_error: 1.1875 - val_loss: 1.2606 - val_mean_squared_error: 1.2606
Epoch 3/100
396/396 [==============================] - 0s 53us/step - loss: 1.1563 - mean_squared_error: 1.1563 - val_loss: 1.2201 - val_mean_squared_error: 1.2201
Epoch 4/100
396/396 [==============================] - 0s 55us/step - loss: 1.1315 - mean_squared_error: 1.1315 - val_loss: 1.1852 - val_mean_squared_error: 1.1852
Epoch 5/100
396/396 [==============================] - 0s 66us/step - loss: 1.1111 - mean_squared_error: 1.1111 - val_loss: 1.1585 - val_mean_squared_error: 1.1585
Epoch 6/100
396/396 [==============================] - 0s 68us/step - loss: 1.0940 - mean_squared_error: 1.0940 - val_loss: 1.1334 - val

396/396 [==============================] - 0s 78us/step - loss: 0.9357 - mean_squared_error: 0.9357 - val_loss: 0.9493 - val_mean_squared_error: 0.9493
Epoch 51/100
396/396 [==============================] - 0s 76us/step - loss: 0.9357 - mean_squared_error: 0.9357 - val_loss: 0.9485 - val_mean_squared_error: 0.9485
Epoch 52/100
396/396 [==============================] - 0s 81us/step - loss: 0.9345 - mean_squared_error: 0.9345 - val_loss: 0.9475 - val_mean_squared_error: 0.9475
Epoch 53/100
396/396 [==============================] - 0s 81us/step - loss: 0.9336 - mean_squared_error: 0.9336 - val_loss: 0.9481 - val_mean_squared_error: 0.9481
Epoch 54/100
396/396 [==============================] - 0s 83us/step - loss: 0.9330 - mean_squared_error: 0.9330 - val_loss: 0.9505 - val_mean_squared_error: 0.9505
Epoch 55/100
396/396 [==============================] - 0s 88us/step - loss: 0.9331 - mean_squared_error: 0.9331 - val_loss: 0.9479 - val_mean_squared_error: 0.9479
Epoch 56/100
396/396 [=

Epoch 100/100
396/396 [==============================] - 0s 76us/step - loss: 0.9171 - mean_squared_error: 0.9171 - val_loss: 0.9436 - val_mean_squared_error: 0.9436


In [77]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [21]:
print(MSE_train)
print(MSE_val)

0.914450642739685
0.9437157484784983


## Adam

In [78]:
np.random.seed(123)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= "Adam" ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 1)

Train on 396 samples, validate on 99 samples
Epoch 1/100
396/396 [==============================] - 1s 1ms/step - loss: 1.2564 - mean_squared_error: 1.2564 - val_loss: 1.3506 - val_mean_squared_error: 1.3506
Epoch 2/100
396/396 [==============================] - 0s 55us/step - loss: 1.2088 - mean_squared_error: 1.2088 - val_loss: 1.2934 - val_mean_squared_error: 1.2934
Epoch 3/100
396/396 [==============================] - 0s 50us/step - loss: 1.1732 - mean_squared_error: 1.1732 - val_loss: 1.2459 - val_mean_squared_error: 1.2459
Epoch 4/100
396/396 [==============================] - 0s 60us/step - loss: 1.1464 - mean_squared_error: 1.1464 - val_loss: 1.2070 - val_mean_squared_error: 1.2070
Epoch 5/100
396/396 [==============================] - 0s 65us/step - loss: 1.1238 - mean_squared_error: 1.1238 - val_loss: 1.1748 - val_mean_squared_error: 1.1748
Epoch 6/100
396/396 [==============================] - 0s 68us/step - loss: 1.1043 - mean_squared_error: 1.1043 - val_loss: 1.1507 - val

396/396 [==============================] - 0s 91us/step - loss: 0.9346 - mean_squared_error: 0.9346 - val_loss: 0.9411 - val_mean_squared_error: 0.9411
Epoch 51/100
396/396 [==============================] - 0s 88us/step - loss: 0.9347 - mean_squared_error: 0.9347 - val_loss: 0.9422 - val_mean_squared_error: 0.9422
Epoch 52/100
396/396 [==============================] - 0s 78us/step - loss: 0.9334 - mean_squared_error: 0.9334 - val_loss: 0.9408 - val_mean_squared_error: 0.9408
Epoch 53/100
396/396 [==============================] - 0s 78us/step - loss: 0.9327 - mean_squared_error: 0.9327 - val_loss: 0.9394 - val_mean_squared_error: 0.9394
Epoch 54/100
396/396 [==============================] - 0s 78us/step - loss: 0.9322 - mean_squared_error: 0.9322 - val_loss: 0.9404 - val_mean_squared_error: 0.9404
Epoch 55/100
396/396 [==============================] - 0s 93us/step - loss: 0.9333 - mean_squared_error: 0.9333 - val_loss: 0.9428 - val_mean_squared_error: 0.9428
Epoch 56/100
396/396 [=

Epoch 100/100
396/396 [==============================] - 0s 81us/step - loss: 0.9122 - mean_squared_error: 0.9122 - val_loss: 0.9440 - val_mean_squared_error: 0.9440


In [79]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [80]:
print(MSE_train)
print(MSE_val)

0.9114701248328614
0.9440459674134032


## Learning Rate Decay with Momentum


In [81]:
np.random.seed(123)
sgd = optimizers.SGD(lr=0.03, decay=0.0001, momentum=0.9)
model = Sequential()
model.add(layers.Dense(8, input_dim=12, activation='relu'))
model.add(layers.Dense(1, activation = 'linear'))

model.compile(optimizer= sgd ,loss='mse',metrics=['mse'])
hist = model.fit(X_train, Y_train, batch_size=32, 
                 epochs=100, validation_data = (X_val, Y_val), verbose = 0)

In [82]:
pred_train = model.predict(X_train).reshape(-1)
pred_val = model.predict(X_val).reshape(-1)

MSE_train = np.mean((pred_train-Y_train)**2)
MSE_val = np.mean((pred_val-Y_val)**2)

In [83]:
print(MSE_train)
print(MSE_val)

0.8383352017382003
0.9126765851437523


## Additional Resources
* https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Consumer_complaints.ipynb  

* https://catalog.data.gov/dataset/consumer-complaint-database  

* https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/  

* https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/  

* https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/  

* https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network  

* https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/


## Summary  

In this lab, we began to practice some of the concepts regarding normalization and optimization for neural networks. In the final lab for this section, you'll independently practice these concepts on your own in order to tune a model to predict individuals payments to loans.